In [1]:
#import der benötigten Python Libraries 
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import time
import datetime

import re
import emoji

import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', 500)

#um die häufigsten Wörter im Wahlkampf anzeigen lassen zu können > wird zur tokenisierung etc benötigt
import nltk 
from nltk.corpus import stopwords
from nltk import FreqDist

import spacy

In [2]:
#DataFrames von allen Parteien
#cdu
df_cdu_raw = pd.read_excel("../data/Parteien/@CDU_user_tweets_bearbeitet.xlsx")
df_cdu_raw.rename(columns={'Text': "original_tweets"}, inplace=True)
# Konvertieren des DataSets + umbenennen der Spalte
df_cdu = df_cdu_raw[['Name', 'original_tweets']]

#spd
df_spd_raw = pd.read_excel("../data/Parteien/@spdde_user_tweets_bearbeitet.xlsx")
df_spd_raw.rename(columns={'Text': "original_tweets"}, inplace=True)
df_spd = df_spd_raw[['Name', 'original_tweets']]

#fdp
df_fdp_raw = pd.read_excel("../data/Parteien/@fdp_user_tweets_bearbeitet.xlsx")
df_fdp_raw.rename(columns={'Text': "original_tweets"}, inplace=True)
df_fdp = df_fdp_raw[['Name', 'original_tweets']]

#die Grünen
df_gruene_raw = pd.read_excel("../data/Parteien/@Die_Gruenen_user_tweets_bearbeitet.xlsx")
df_gruene_raw.rename(columns={'Text': "original_tweets"}, inplace=True)
df_gruene = df_gruene_raw[['Name', 'original_tweets']]

#Die Linken 
df_linke_raw = pd.read_excel("../data/Parteien/@dieLinke_user_tweets_bearbeitet.xlsx")
df_linke_raw.rename(columns={'Text': "original_tweets"}, inplace=True)
df_linke = df_linke_raw[['Name', 'original_tweets']]

#AfD
df_afd_raw = pd.read_excel("../data/Parteien/@AfD_user_tweets_bearbeitet.xlsx")
df_afd_raw.rename(columns={'Text': "original_tweets"}, inplace=True)
df_afd = df_afd_raw[['Name', 'original_tweets']]

#CSU
df_csu_raw = pd.read_excel("../data/Parteien/@CSU_user_tweets_bearbeitet.xlsx")
df_csu_raw.rename(columns={'Text': "original_tweets"}, inplace=True)
df_csu = df_csu_raw[['Name', 'original_tweets']]




In [3]:
#Zusammenfügen der dataframes für Gesamtanalayse
#df_all_parties = pd.concat([df_cdu_raw, df_spd_raw], axis=0)
df_all_parties = df_cdu.append(df_spd, ignore_index=True)
df_all_parties = df_all_parties.append(df_fdp, ignore_index=True)
df_all_parties = df_all_parties.append(df_gruene, ignore_index=True)
df_all_parties = df_all_parties.append(df_linke, ignore_index=True)
df_all_parties = df_all_parties.append(df_afd, ignore_index=True)
df_all_parties = df_all_parties.append(df_csu, ignore_index=True)

tweets_df_clean = df_all_parties

In [108]:
import string 

def clean_text(text):
#konvertiert text zu lowercase  
 text = text.lower()
#löscht text in eckigen Klammern
 text = re.sub(r'\[.*?\]', '', text)
#löscht Punktuation + Links + @Zeichen 
 text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text) 
#löscht Wörter die Nummern enthalten
 text = re.sub(r'\w*\d\w*', '', text)
#löscht Anführungsstriche
 text = re.sub(r'„', '', text)
 text = re.sub(r'“', '', text)
 return text

def give_emoji_free_text(text): #Emojis werden aus den tweets entfernt 
    return emoji.get_emoji_regexp().sub(r'',text)

tweets_df_clean['emoji_free_tweets'] = tweets_df_clean['original_tweets'].apply(give_emoji_free_text)

#Create a new column with emji free text 
tweets_df_clean['clean_tweets'] = tweets_df_clean['emoji_free_tweets'].apply(clean_text)                               
                               
tweets_df_clean

,Name,original_tweets,emoji_free_tweets,clean_tweets
0,CDU Deutschlands,".@ArminLaschet in der #BerlinerRunde: „Die Zeit die vor uns liegt, braucht als erstes wirtschaftliche Stärke &amp; Stabilität, marktwirtschaftliche Lösungen und einen großen Schub in der Verwaltung des Landes, der Digitalisierung sowie im Klimaschutz!“",".@ArminLaschet in der #BerlinerRunde: „Die Zeit die vor uns liegt, braucht als erstes wirtschaftliche Stärke &amp; Stabilität, marktwirtschaftliche Lösungen und einen großen Schub in der Verwaltung des Landes, der Digitalisierung sowie im Klimaschutz!“",arminlaschet in der berlinerrunde die zeit die vor uns liegt braucht als erstes wirtschaftliche stärke amp stabilität marktwirtschaftliche lösungen und einen großen schub in der verwaltung des landes der digitalisierung sowie im klimaschutz
1,CDU Deutschlands,Jetzt live: Statement von @ArminLaschet am Wahlabend der #btw21.\n https://t.co/BjbG8lb8aG,Jetzt live: Statement von @ArminLaschet am Wahlabend der #btw21.\n https://t.co/BjbG8lb8aG,jetzt live statement von arminlaschet am wahlabend der \n
2,CDU Deutschlands,"Wir danken allen Wählerinnen und Wählern, die uns heute ihr Vertrauen geschenkt haben.","Wir danken allen Wählerinnen und Wählern, die uns heute ihr Vertrauen geschenkt haben.",wir danken allen wählerinnen und wählern die uns heute ihr vertrauen geschenkt haben
3,CDU Deutschlands,"An alle Wahlkämpferinnen und Wahlkämpfer, die bei Regen, Sonne oder Sturm, bei Tag und Nacht, von Tür zu Tür, auf den Marktplätzen oder in den sozialen Medien für eine starke CDU und @ArminLaschet als Bundeskanzler gekämpft haben.","An alle Wahlkämpferinnen und Wahlkämpfer, die bei Regen, Sonne oder Sturm, bei Tag und Nacht, von Tür zu Tür, auf den Marktplätzen oder in den sozialen Medien für eine starke CDU und @ArminLaschet als Bundeskanzler gekämpft haben.",an alle wahlkämpferinnen und wahlkämpfer die bei regen sonne oder sturm bei tag und nacht von tür zu tür auf den marktplätzen oder in den sozialen medien für eine starke cdu und arminlaschet als bundeskanzler gekämpft haben
4,CDU Deutschlands,"Es ist ein Kopf-an-Kopf-Rennen und wird ein langer Wahlabend. Zum jetzigen Zeitpunkt kann man noch nicht sagen, wer am Ende die Nase vorn haben wird.\nJetzt möchten wir erst einmal Danke sagen: https://t.co/G87SxsFhQd","Es ist ein Kopf-an-Kopf-Rennen und wird ein langer Wahlabend. Zum jetzigen Zeitpunkt kann man noch nicht sagen, wer am Ende die Nase vorn haben wird.\nJetzt möchten wir erst einmal Danke sagen: https://t.co/G87SxsFhQd",es ist ein kopfankopfrennen und wird ein langer wahlabend zum jetzigen zeitpunkt kann man noch nicht sagen wer am ende die nase vorn haben wird\njetzt möchten wir erst einmal danke sagen
...,...,...,...,...
3367,CSU,Gedenkminute im Stadion für die Opfer der entsetzlichen Messerattacke in #Würzburg. Wir sind in Gedanken bei den Familien. https://t.co/vEginx1LxT,Gedenkminute im Stadion für die Opfer der entsetzlichen Messerattacke in #Würzburg. Wir sind in Gedanken bei den Familien. https://t.co/vEginx1LxT,gedenkminute im stadion für die opfer der entsetzlichen messerattacke in würzburg wir sind in gedanken bei den familien
3368,CSU,🔵 Jetzt LIVE: die Aufstellung der #CSU-Liste für die Bundestagswahl aus dem Max-Morlock-Stadtion in Nürnberg. ⬇️ https://t.co/yk6Nvr90xW,Jetzt LIVE: die Aufstellung der #CSU-Liste für die Bundestagswahl aus dem Max-Morlock-Stadtion in Nürnberg. https://t.co/yk6Nvr90xW,jetzt live die aufstellung der csuliste für die bundestagswahl aus dem maxmorlockstadtion in nürnberg
3369,CSU,".@Markus_Soeder: Ich danke der #Polizei für ihren schnellen Einsatz. Besonders beeindruckt hat mich das Engagement der Bürger! Wir stehen nicht nur den Betroffenen bei, sondern werden alles tun, um so etwas in Zukunft zu verhindern! #Würzburg",".@Markus_Soeder: Ich danke der #Polizei für ihren schnellen Einsatz. Besonders beeindruckt hat mich das Engagement der Bürger! Wir stehen nicht nur den Betroffenen bei, sondern werden alles tun, um so et

In [109]:
data = list(tweets_df_clean.loc[:, "clean_tweets"].values) ## data ist ein doc

In [110]:
# Die Wörter werden lemmatisiert -> ihr Ursprungsform wird betracht (ungebeugt)
# durch den Parameter "allowed_postags" filtern wir den Text und lassen nur Substatntive und Adjektive ausgeben
def lemmatization(texts, allowed_postags=["NOUN","ADJ"]):
    nlp = spacy.load("de_core_news_lg", disable=["parser", "ner"]) 
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:120])

berlinerrunde zeit erst wirtschaftlich stärken stabilität marktwirtschaftliche lösungen groß schub verwaltung landes dig


In [111]:
tweets_df_clean['clean_tweets_noun'] = lemmatized_texts

In [112]:
tweets_df_clean

,Name,original_tweets,emoji_free_tweets,clean_tweets,clean_tweets_noun
0,CDU Deutschlands,".@ArminLaschet in der #BerlinerRunde: „Die Zeit die vor uns liegt, braucht als erstes wirtschaftliche Stärke &amp; Stabilität, marktwirtschaftliche Lösungen und einen großen Schub in der Verwaltung des Landes, der Digitalisierung sowie im Klimaschutz!“",".@ArminLaschet in der #BerlinerRunde: „Die Zeit die vor uns liegt, braucht als erstes wirtschaftliche Stärke &amp; Stabilität, marktwirtschaftliche Lösungen und einen großen Schub in der Verwaltung des Landes, der Digitalisierung sowie im Klimaschutz!“",arminlaschet in der berlinerrunde die zeit die vor uns liegt braucht als erstes wirtschaftliche stärke amp stabilität marktwirtschaftliche lösungen und einen großen schub in der verwaltung des landes der digitalisierung sowie im klimaschutz,berlinerrunde zeit erst wirtschaftlich stärken stabilität marktwirtschaftliche lösungen groß schub verwaltung landes digitalisierung klimaschutz
1,CDU Deutschlands,Jetzt live: Statement von @ArminLaschet am Wahlabend der #btw21.\n https://t.co/BjbG8lb8aG,Jetzt live: Statement von @ArminLaschet am Wahlabend der #btw21.\n https://t.co/BjbG8lb8aG,jetzt live statement von arminlaschet am wahlabend der \n,statement
2,CDU Deutschlands,"Wir danken allen Wählerinnen und Wählern, die uns heute ihr Vertrauen geschenkt haben.","Wir danken allen Wählerinnen und Wählern, die uns heute ihr Vertrauen geschenkt haben.",wir danken allen wählerinnen und wählern die uns heute ihr vertrauen geschenkt haben,wählerinnen wählern vertrauen
3,CDU Deutschlands,"An alle Wahlkämpferinnen und Wahlkämpfer, die bei Regen, Sonne oder Sturm, bei Tag und Nacht, von Tür zu Tür, auf den Marktplätzen oder in den sozialen Medien für eine starke CDU und @ArminLaschet als Bundeskanzler gekämpft haben.","An alle Wahlkämpferinnen und Wahlkämpfer, die bei Regen, Sonne oder Sturm, bei Tag und Nacht, von Tür zu Tür, auf den Marktplätzen oder in den sozialen Medien für eine starke CDU und @ArminLaschet als Bundeskanzler gekämpft haben.",an alle wahlkämpferinnen und wahlkämpfer die bei regen sonne oder sturm bei tag und nacht von tür zu tür auf den marktplätzen oder in den sozialen medien für eine starke cdu und arminlaschet als bundeskanzler gekämpft haben,wahlkämpferinnen wahlkämpfer regen sonne sturm tag nacht tür tür marktplätzen sozial medien starke bundeskanzler
4,CDU Deutschlands,"Es ist ein Kopf-an-Kopf-Rennen und wird ein langer Wahlabend. Zum jetzigen Zeitpunkt kann man noch nicht sagen, wer am Ende die Nase vorn haben wird.\nJetzt möchten wir erst einmal Danke sagen: https://t.co/G87SxsFhQd","Es ist ein Kopf-an-Kopf-Rennen und wird ein langer Wahlabend. Zum jetzigen Zeitpunkt kann man noch nicht sagen, wer am Ende die Nase vorn haben wird.\nJetzt möchten wir erst einmal Danke sagen: https://t.co/G87SxsFhQd",es ist ein kopfankopfrennen und wird ein langer wahlabend zum jetzigen zeitpunkt kann man noch nicht sagen wer am ende die nase vorn haben wird\njetzt möchten wir erst einmal danke sagen,kopfankopfrennen lang jetzig zeitpunkt ende nase danke
...,...,...,...,...,...
3367,CSU,Gedenkminute im Stadion für die Opfer der entsetzlichen Messerattacke in #Würzburg. Wir sind in Gedanken bei den Familien. https://t.co/vEginx1LxT,Gedenkminute im Stadion für die Opfer der entsetzlichen Messerattacke in #Würzburg. Wir sind in Gedanken bei den Familien. https://t.co/vEginx1LxT,gedenkminute im stadion für die opfer der entsetzlichen messerattacke in würzburg wir sind in gedanken bei den familien,gedenkminute stadion opfer entsetzlich messerattacke gedanken familien
3368,CSU,🔵 Jetzt LIVE: die Aufstellung der #CSU-Liste für die Bundestagswahl aus dem Max-Morlock-Stadtion in Nürnberg. ⬇️ https://t.co/yk6Nvr90xW,Jetzt LIVE: die Aufstellung der #CSU-Liste für die Bundestagswahl aus dem Max-Morlock-Stadtion in Nürnberg. https://t.co/yk6Nvr90xW,jetzt live die aufstellung der csuliste für die bundestagswahl aus dem maxmorlockstadtion in nürnberg,aufstellung 

In [124]:
#stopwords werden entfernt 
#sätze werden in einzelne Wörter heruntergebochen 

import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('german')
stop_words.extend(['amp','mehr erfahren','erfahren','arminlaschet','wegenmorgen','olafscholz','olaf','scholz','scholzpacktdasan','eskensaskia','nowabofm','larsklingbeil','abaerbock','vielzutun','clindner','httpstcouelfxzluyy','janinewissler','link','joergschindler','afd','tinochrupalla','markusblume','markussoeder'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = tweets_df_clean.clean_tweets_noun.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/philipp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [125]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(tweets_df_clean.clean_tweets_noun, 10)
unigram = pd.DataFrame(common_words, columns = ['unigram' , 'count'])

In [126]:
unigram

,unigram,count
0,menschen,276
1,klimaschutz,233
2,gut,208
3,land,203
4,uhr,132
5,politik,126
6,zukunft,120
7,groß,111
8,union,110
9,euro,106


In [127]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2,2),stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(tweets_df_clean.clean_tweets_noun, 10)
bigram = pd.DataFrame(common_words, columns = ['bigram' , 'count'])

In [128]:
bigram

,bigram,count
0,mittler einkommen,26
1,erneuerbaren energien,25
2,mindestlohn euro,25
3,erneuerbare energien,24
4,herzlich glückwunsch,19
5,ausbau erneuerbaren,16
6,september briefwahl,15
7,nächst bundesregierung,15
8,menschen land,14
9,nächst bundeskanzler,14


In [129]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3,3),stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(tweets_df_clean.clean_tweets_noun, 10)
trigram = pd.DataFrame(common_words, columns = ['trigram' , 'count'])

In [130]:
trigram

,trigram,count
0,kleine mittler einkommen,12
1,gut sicher jobs,10
2,ausbau erneuerbaren energien,10
3,politik verloren zeit,10
4,tage afdwochenendpodcast spannend,9
5,afdwochenendpodcast spannend themen,9
6,jahre spitz erfolg,9
7,klein mittler einkommen,8
8,schluss politik verloren,8
9,ausgabe podcast tage,8


In [131]:
from sklearn.decomposition import LatentDirichletAllocation
vectorizer = CountVectorizer(
                        analyzer='word', 
                        ngram_range=(1,2),#bigramme werden berücksichtigt 
                        max_df=0.8,
                        min_df=2,# minimales Vorkommen eines Wortes
                        stop_words= stop_words,#stopwords werden entfernt 
                        lowercase=True #lowercase 
                        )
data_matrix = vectorizer.fit_transform(tweets_df_clean.clean_tweets_noun)
data_matrix

<3372x4022 sparse matrix of type '<class 'numpy.int64'>'
	with 23141 stored elements in Compressed Sparse Row format>

In [148]:
lda_model = LatentDirichletAllocation(
doc_topic_prior=0,
n_components=5, # Nummer von Topics
learning_method='batch',
random_state=100,       
n_jobs = -1  # Alle verfügbaren CPUs werden verwendet 
                                     )
lda_output = lda_model.fit_transform(data_matrix)

In [149]:
for i,topic in enumerate(lda_model.components_):
    print(f'Top 10 words for topic #{i}:')
    print([vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')


Top 10 words for topic #0:
['arbeitsplätze', 'einkommen', 'euro', 'grün', 'union', 'klimaschutz', 'gut', 'hoch', 'steuern', 'krise']


Top 10 words for topic #1:
['wohnungen', 'klaren', 'zukunft', 'gut', 'land', 'nächst', 'jahr', 'menschen', 'jahren', 'groß']


Top 10 words for topic #2:
['euro', 'weit', 'wirtschaft', 'hoch', 'stark', 'mindestlohn', 'millionen', 'gut', 'uhr', 'menschen']


Top 10 words for topic #3:
['groß', 'zeit', 'gut', 'bundestagswahl', 'klimaschutz', 'bundestag', 'kinder', 'menschen', 'september', 'land']


Top 10 words for topic #4:
['landwirtschaft', 'sicherheit', 'erneuerbaren', 'livestream', 'politik', 'gut', 'bundesregierung', 'ausbau', 'menschen', 'klimaschutz']




In [147]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_matrix, vectorizer, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
1      150.222153  -72.269554       1        1  22.415491
2       51.163364 -159.968018       2        1  21.032324
3      -73.144554   21.523815       3        1  20.891971
0      -66.410133 -110.606438       4        1  18.331797
4       54.417324   12.443444       5        1  17.328417, topic_info=                 Term        Freq       Total Category  logprob  loglift
1854      klimaschutz  199.000000  199.000000  Default  30.0000  30.0000
2994       sicherheit   56.000000   56.000000  Default  29.0000  29.0000
1728  kanzlerkandidat   41.000000   41.000000  Default  28.0000  28.0000
1779             klar   34.000000   34.000000  Default  27.0000  27.0000
2031             land  174.000000  174.000000  Default  26.0000  26.0000
...               ...         ...         ...      ...      ...      ...
3412           triell   14.726179   85.495512   Topic5  -5.6365  -0.0060
1221     gesellschaft   13.757597   65.083417   Topic5  -5.7046   0.1987
2892     schlussrunde   13.345238   67.470616   Topic5  -5.7350   0.1323
423           bildung   12.475707   38.556513   Topic5  -5.8024   0.6245
2433              neu   12.932440   62.375230   Topic5  -5.7664   0.1794

[355 rows x 6 columns], token_table=      Topic      Freq                          Term
term                                               
31        5  0.881185                         abzug
49        3  0.932456           afdwochenendpodcast
50        3  0.932456  afdwochenendpodcast spannend
108       4  0.840358                      angebote
126       1  0.947050                       ansagen
...     ...       ...                           ...
3946      2  0.202427                       zukunft
3946      3  0.250624                       zukunft
3946      4  0.134951                       zukunft
3991      2  0.117369                        zweite
3991      5  0.821580                        zweite

[594 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 4, 1, 5])